In [ ]:
"""
def fetch_aave_reserves_defillama(chain: str) -> pd.DataFrame:
    #Fetch Aave data from Defillama API
    
    chain_mapping = {
        'ethereum': 'ethereum',
        'polygon': 'polygon', 
        'avalanche': 'avalanche',
        'optimism': 'optimism',
        'arbitrum': 'arbitrum'
    }
    
    try:
        url = "https://yields.llama.fi/pools"
        response = requests.get(url, timeout=30)
        data = response.json()
        
        # Filter for Aave V3 pools
        aave_pools = [
            pool for pool in data['data'] 
            if 'aave' in pool['project'].lower() and 'v3' in pool['project'].lower()
            and pool['chain'] == chain_mapping.get(chain, chain)
        ]
        
        df = pd.DataFrame(aave_pools)
        print(f"✅ Found {len(df)} Aave V3 pools on {chain} via Defillama")
        return df
        
    except Exception as e:
        print(f"❌ Defillama API failed: {e}")
        print(pd.DataFrame())
"""

In [ ]:
"""
from web3 import Web3
import pandas as pd

def fetch_aave_reserves_simple(chain: str) -> pd.DataFrame:
    #Simple version focusing on core risk metrics
    
    rpc_endpoints = {
        'ethereum': 'https://eth.llamarpc.com',
        'polygon': 'https://polygon-rpc.com'
    }
    
    if chain not in rpc_endpoints:
        return pd.DataFrame()
    
    try:
        w3 = Web3(Web3.HTTPProvider(rpc_endpoints[chain]))
        
        pool_addresses = {
            'ethereum': '0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2',
            'polygon': '0x794a61358D6845594F94dc1DB02A252b5b4814aD'
        }
        
        pool_abi = [
            {
                "inputs": [],
                "name": "getReservesList",
                "outputs": [{"internalType": "address[]", "name": "", "type": "address[]"}],
                "stateMutability": "view",
                "type": "function"
            },
            {
                "inputs": [{"internalType": "address", "name": "asset", "type": "address"}],
                "name": "getReserveData",
                "outputs": [
                    {"internalType": "uint256", "name": "configuration", "type": "uint256"},
                    {"internalType": "uint128", "name": "liquidityIndex", "type": "uint128"},
                    {"internalType": "uint128", "name": "variableBorrowIndex", "type": "uint128"},
                    {"internalType": "uint128", "name": "currentLiquidityRate", "type": "uint128"},
                    {"internalType": "uint128", "name": "currentVariableBorrowRate", "type": "uint128"},
                    {"internalType": "uint128", "name": "currentStableBorrowRate", "type": "uint128"},
                    {"internalType": "uint40", "name": "lastUpdateTimestamp", "type": "uint40"},
                    {"internalType": "uint16", "name": "id", "type": "uint16"},
                    {"internalType": "address", "name": "aTokenAddress", "type": "address"},
                    {"internalType": "address", "name": "stableDebtTokenAddress", "type": "address"},
                    {"internalType": "address", "name": "variableDebtTokenAddress", "type": "address"},
                    {"internalType": "address", "name": "interestRateStrategyAddress", "type": "address"},
                    {"internalType": "uint128", "name": "accruedToTreasury", "type": "uint128"},
                    {"internalType": "uint128", "name": "unbacked", "type": "uint128"},
                    {"internalType": "uint128", "name": "isolationModeTotalDebt", "type": "uint128"}
                ],
                "stateMutability": "view",
                "type": "function"
            }
        ]
        
        pool_contract = w3.eth.contract(
            address=Web3.to_checksum_address(pool_addresses[chain]),
            abi=pool_abi
        )
        
        reserves_list = pool_contract.functions.getReservesList().call()
        reserves_data = []
        
        for reserve in reserves_list[:5]:  # Just first 5 for testing
            try:
                reserve_data = pool_contract.functions.getReserveData(reserve).call()
                
                reserve_info = {
                    'address': reserve,
                    'liquidityRate': reserve_data[3] / 1e27,
                    'variableBorrowRate': reserve_data[4] / 1e27,
                    'stableBorrowRate': reserve_data[5] / 1e27,
                    'lastUpdateTimestamp': reserve_data[6]
                }
                reserves_data.append(reserve_info)
                
            except Exception as e:
                continue
        
        return pd.DataFrame(reserves_data)
        
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

# Test the simple version
if __name__ == "__main__":
    df = fetch_aave_reserves_simple("ethereum")
    if not df.empty:
        print("✅ Success! Data fetched:")
        print(df.head())
"""

✅ Success! Data fetched:
                                      address  liquidityRate  \
0  0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2       1.082273   
1  0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0       1.011080   
2  0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599       1.023912   
3  0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48       1.195909   
4  0x6B175474E89094C44Da98b954EedeAC495271d0F       1.209954   

   variableBorrowRate  stableBorrowRate  lastUpdateTimestamp  
0            0.023041               0.0           1759480151  
1            0.002069               0.0           1759478471  
2            0.002087               0.0           1759479767  
3            0.054321               0.0           1759480151  
4            0.062691               0.0           1759479479  


In [36]:
from web3 import Web3
import pandas as pd
import requests

def fetch_complete_aave_reserves_working(chain: str) -> pd.DataFrame:
    """Complete working version with all risk parameters"""
    
    rpc_endpoints = {
        'ethereum': 'https://eth.llamarpc.com',
        'polygon': 'https://polygon-rpc.com',
        'arbitrum': 'https://arb1.arbitrum.io/rpc',
        'optimism': 'https://mainnet.optimism.io',
        'avalanche': 'https://api.avax.network/ext/bc/C/rpc'
    }
    
    if chain not in rpc_endpoints:
        print(f"⚠️ Chain {chain} not supported")
        return pd.DataFrame()
    
    try:
        print(f"🔗 Connecting to {chain} RPC...")
        w3 = Web3(Web3.HTTPProvider(rpc_endpoints[chain]))
        
        if not w3.is_connected():
            print("❌ Failed to connect to RPC")
            return pd.DataFrame()
            
        print(f"✅ Connected to {chain} at block {w3.eth.block_number}")

        # Aave V3 Pool contract addresses
        pool_addresses = {
            'ethereum': '0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2',
            'polygon': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',
            'arbitrum': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',
            'optimism': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',
            'avalanche': '0x794a61358D6845594F94dc1DB02A252b5b4814aD'
        }
        
        # ABI for getReservesList and getReserveData only
        pool_abi = [
            {
                "inputs": [],
                "name": "getReservesList",
                "outputs": [{"internalType": "address[]", "name": "", "type": "address[]"}],
                "stateMutability": "view",
                "type": "function"
            },
            {
                "inputs": [{"internalType": "address", "name": "asset", "type": "address"}],
                "name": "getReserveData",
                "outputs": [
                    {"internalType": "uint256", "name": "configuration", "type": "uint256"},
                    {"internalType": "uint128", "name": "liquidityIndex", "type": "uint128"},
                    {"internalType": "uint128", "name": "variableBorrowIndex", "type": "uint128"},
                    {"internalType": "uint128", "name": "currentLiquidityRate", "type": "uint128"},
                    {"internalType": "uint128", "name": "currentVariableBorrowRate", "type": "uint128"},
                    {"internalType": "uint128", "name": "currentStableBorrowRate", "type": "uint128"},
                    {"internalType": "uint40", "name": "lastUpdateTimestamp", "type": "uint40"},
                    {"internalType": "uint16", "name": "id", "type": "uint16"},
                    {"internalType": "address", "name": "aTokenAddress", "type": "address"},
                    {"internalType": "address", "name": "stableDebtTokenAddress", "type": "address"},
                    {"internalType": "address", "name": "variableDebtTokenAddress", "type": "address"},
                    {"internalType": "address", "name": "interestRateStrategyAddress", "type": "address"},
                    {"internalType": "uint128", "name": "accruedToTreasury", "type": "uint128"},
                    {"internalType": "uint128", "name": "unbacked", "type": "uint128"},
                    {"internalType": "uint128", "name": "isolationModeTotalDebt", "type": "uint128"}
                ],
                "stateMutability": "view",
                "type": "function"
            }
        ]
        
        pool_address = pool_addresses[chain]
        pool_contract = w3.eth.contract(
            address=Web3.to_checksum_address(pool_address),
            abi=pool_abi
        )
        
        # Get list of reserves
        print("📋 Getting reserves list...")
        reserves_list = pool_contract.functions.getReservesList().call()
        print(f"📊 Found {len(reserves_list)} reserves")
        
        # Get token symbols and decimals
        def get_token_info(address):
            try:
                erc20_abi = [
                    {"constant": True, "inputs": [], "name": "symbol", "outputs": [{"name": "", "type": "string"}], "type": "function"},
                    {"constant": True, "inputs": [], "name": "decimals", "outputs": [{"name": "", "type": "uint8"}], "type": "function"},
                    {"constant": True, "inputs": [], "name": "name", "outputs": [{"name": "", "type": "string"}], "type": "function"}
                ]
                token_contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=erc20_abi)
                symbol = token_contract.functions.symbol().call()
                decimals = token_contract.functions.decimals().call()
                name = token_contract.functions.name().call()
                return symbol, decimals, name
            except:
                return "UNKNOWN", 18, "Unknown"
        
        # Get data for all reserves
        reserves_data = []
        successful_reserves = 0
        
        for i, reserve in enumerate(reserves_list):
            try:
                if i % 10 == 0:
                    print(f"🔍 Processing reserve {i+1}/{len(reserves_list)}...")
                
                # Get reserve data
                reserve_data = pool_contract.functions.getReserveData(reserve).call()
                
                # Get token info
                symbol, decimals, name = get_token_info(reserve)
                
                # Extract configuration data (bitmask)
                configuration = reserve_data[0]
                
                # Decode configuration (bitmask positions for Aave V3)
                # Reference: https://docs.aave.com/developers/core-contracts/pool#getreservedata
                is_active = (configuration >> 56) & 1 == 1
                is_frozen = (configuration >> 57) & 1 == 1
                borrowing_enabled = (configuration >> 58) & 1 == 1
                stable_borrowing_enabled = (configuration >> 59) & 1 == 1
                
                # Decode LTV, liquidation threshold, and liquidation bonus from configuration
                ltv = (configuration & ((1 << 16) - 1)) / 10000  # bits 0-15
                liquidation_threshold = ((configuration >> 16) & ((1 << 16) - 1)) / 10000  # bits 16-31
                liquidation_bonus = ((configuration >> 32) & ((1 << 16) - 1)) / 10000  # bits 32-47
                
                reserve_info = {
                    'address': reserve,
                    'symbol': symbol,
                    'name': name,
                    'decimals': decimals,
                    'liquidityRate': reserve_data[3] / 1e27,  # APY
                    'variableBorrowRate': reserve_data[4] / 1e27,  # APY
                    'stableBorrowRate': reserve_data[5] / 1e27,  # APY
                    'ltv': ltv,
                    'liquidationThreshold': liquidation_threshold,
                    'liquidationBonus': liquidation_bonus,
                    'isActive': is_active,
                    'isFrozen': is_frozen,
                    'borrowingEnabled': borrowing_enabled,
                    'stableBorrowingEnabled': stable_borrowing_enabled,
                    'liquidityIndex': reserve_data[1] / 1e27,
                    'variableBorrowIndex': reserve_data[2] / 1e27,
                    'lastUpdateTimestamp': reserve_data[6],
                    'aTokenAddress': reserve_data[8],
                    'variableDebtTokenAddress': reserve_data[10]
                }
                reserves_data.append(reserve_info)
                successful_reserves += 1
                
            except Exception as e:
                print(f"⚠️ Failed to get data for reserve {reserve}: {e}")
                continue
        
        df = pd.DataFrame(reserves_data)
        print(f"✅ Successfully processed {successful_reserves}/{len(reserves_list)} reserves")
        
        if successful_reserves > 0:
            # Calculate risk metrics
            df['utilization_rate'] = df['variableBorrowRate'] / (df['liquidityRate'] + 1e-18)
            df['safety_margin'] = df['liquidationThreshold'] - df['ltv']
            df['borrow_apy'] = df['variableBorrowRate'] * 100
            df['supply_apy'] = df['liquidityRate'] * 100
            
            print(f"🎯 Risk data calculated for {len(df)} reserves")
        
        return df
        
    except Exception as e:
        print(f"❌ RPC method failed: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

# Test the working version
if __name__ == "__main__":
    print("🚀 Fetching Aave V3 reserves with complete risk parameters...")
    df = fetch_complete_aave_reserves_working("ethereum")
    
    if not df.empty:
        print(f"\n🎯 COMPLETE RISK MONITORING DATA - {len(df)} RESERVES")
        print("=" * 100)
        
        # Display comprehensive risk metrics
        risk_columns = [
            'symbol', 'liquidityRate', 'variableBorrowRate', 'ltv', 
            'liquidationThreshold', 'utilization_rate', 'safety_margin',
            'isActive', 'borrowingEnabled'
        ]
        
        print(df[risk_columns].head(15).to_string(index=False, float_format='%.4f'))
        
        print(f"\n📊 RISK SUMMARY:")
        print(f"• Total reserves: {len(df)}")
        print(f"• Highest borrow rate: {df['variableBorrowRate'].max():.2%}")
        print(f"• Average LTV: {df['ltv'].mean():.1%}")
        print(f"• Average liquidation threshold: {df['liquidationThreshold'].mean():.1%}")
        print(f"• Reserves with borrowing enabled: {df['borrowingEnabled'].sum()}")
        print(f"• Frozen reserves: {df['isFrozen'].sum()}")
        
        # Identify high-risk reserves
        high_borrow_risk = df[df['variableBorrowRate'] > 0.10]  # > 10% borrow rate
        if not high_borrow_risk.empty:
            print(f"\n🚨 HIGH BORROW RATE RESERVES (>10%):")
            for _, reserve in high_borrow_risk.iterrows():
                print(f"   • {reserve['symbol']}: {reserve['variableBorrowRate']:.2%} borrow rate")
        
        # Identify low safety margin reserves
        low_safety = df[df['safety_margin'] < 0.05]  # < 5% safety margin
        if not low_safety.empty:
            print(f"\n⚠️ LOW SAFETY MARGIN RESERVES (<5% margin):")
            for _, reserve in low_safety.iterrows():
                print(f"   • {reserve['symbol']}: {reserve['safety_margin']:.1%} margin (LTV: {reserve['ltv']:.1%}, Liq: {reserve['liquidationThreshold']:.1%})")
        
        # Identify high utilization reserves
        high_utilization = df[df['utilization_rate'] > 0.80]  # > 80% utilization
        if not high_utilization.empty:
            print(f"\n🔴 HIGH UTILIZATION RESERVES (>80%):")
            for _, reserve in high_utilization.iterrows():
                print(f"   • {reserve['symbol']}: {reserve['utilization_rate']:.1%} utilization")
    
    else:
        print("❌ No data fetched")

🚀 Fetching Aave V3 reserves with complete risk parameters...
🔗 Connecting to ethereum RPC...
✅ Connected to ethereum at block 23496671
📋 Getting reserves list...
📊 Found 55 reserves
🔍 Processing reserve 1/55...
🔍 Processing reserve 11/55...
🔍 Processing reserve 21/55...
🔍 Processing reserve 31/55...
🔍 Processing reserve 41/55...
🔍 Processing reserve 51/55...
✅ Successfully processed 55/55 reserves
🎯 Risk data calculated for 55 reserves

🎯 COMPLETE RISK MONITORING DATA - 55 RESERVES
 symbol  liquidityRate  variableBorrowRate    ltv  liquidationThreshold  utilization_rate  safety_margin  isActive  borrowingEnabled
   WETH         1.0823              0.0230 0.8050                0.8300            0.0213         0.0250      True              True
 wstETH         1.0111              0.0021 0.7850                0.8100            0.0020         0.0250      True              True
   WBTC         1.0239              0.0021 0.7300                0.7800            0.0020         0.0500      True

In [32]:
def fetch_all_aave_chains():
    """Fetch Aave V3 reserves from all supported chains"""
    
    chains = ['ethereum', 'polygon', 'arbitrum', 'optimism', 'avalanche']
    all_chain_data = {}
    
    for chain in chains:
        print(f"\n{'='*60}")
        print(f"🔄 PROCESSING {chain.upper()}")
        print(f"{'='*60}")
        
        df = fetch_complete_aave_reserves_working(chain)
        
        if not df.empty:
            all_chain_data[chain] = df
            
            # Print chain-specific risk summary
            print(f"\n📊 {chain.upper()} RISK SUMMARY:")
            print(f"• Reserves: {len(df)}")
            print(f"• Highest borrow rate: {df['variableBorrowRate'].max():.2%}")
            print(f"• Average LTV: {df['ltv'].mean():.1%}")
            print(f"• Borrowing enabled: {df['borrowingEnabled'].sum()}")
            print(f"• Frozen reserves: {df['isFrozen'].sum()}")
            
            # High risk alerts for this chain
            high_risk = df[df['variableBorrowRate'] > 0.10]
            if not high_risk.empty:
                print(f"🚨 High borrow rate: {', '.join(high_risk['symbol'].tolist())}")
                
        else:
            print(f"❌ No data for {chain}")
    
    return all_chain_data

def generate_cross_chain_risk_report(all_chain_data):
    """Generate comprehensive cross-chain risk analysis"""
    
    print(f"\n{'='*80}")
    print(f"🌐 CROSS-CHAIN AAVE V3 RISK REPORT")
    print(f"{'='*80}")
    
    for chain, df in all_chain_data.items():
        if not df.empty:
            print(f"\n📈 {chain.upper():<12} | Reserves: {len(df):<3} | Avg LTV: {df['ltv'].mean():5.1%} | "
                  f"Max Borrow: {df['variableBorrowRate'].max():5.1%} | "
                  f"High Risk: {len(df[df['variableBorrowRate'] > 0.10]):<2}")
    
    # Cross-chain high risk analysis
    print(f"\n🔴 CROSS-CHAIN HIGH RISK RESERVES (>10% borrow rate):")
    high_risk_all = []
    
    for chain, df in all_chain_data.items():
        if not df.empty:
            high_risk = df[df['variableBorrowRate'] > 0.10]
            for _, reserve in high_risk.iterrows():
                high_risk_all.append({
                    'chain': chain,
                    'symbol': reserve['symbol'],
                    'borrow_rate': reserve['variableBorrowRate'],
                    'ltv': reserve['ltv']
                })
    
    if high_risk_all:
        for risk in sorted(high_risk_all, key=lambda x: x['borrow_rate'], reverse=True):
            print(f"   • {risk['chain']:>10}: {risk['symbol']:<8} {risk['borrow_rate']:>6.1%} borrow rate, LTV: {risk['ltv']:4.1%}")
    else:
        print("   • No high-risk reserves across all chains")

# Run multi-chain analysis
if __name__ == "__main__":
    print("🌐 STARTING MULTI-CHAIN AAVE V3 RISK MONITORING...")
    
    all_data = fetch_all_aave_chains()
    
    if all_data:
        generate_cross_chain_risk_report(all_data)
        
        # Save data for further analysis
        for chain, df in all_data.items():
            if not df.empty:
                filename = f"aave_v3_risk_{chain}.csv"
                df.to_csv(filename, index=False)
                print(f"💾 {chain} data saved to {filename}")
                
        print(f"\n✅ Multi-chain monitoring complete! Processed {len(all_data)} chains")
    else:
        print("❌ No data collected from any chain")

🌐 STARTING MULTI-CHAIN AAVE V3 RISK MONITORING...

🔄 PROCESSING ETHEREUM
🔗 Connecting to ethereum RPC...
✅ Connected to ethereum at block 23496390
📋 Getting reserves list...
📊 Found 55 reserves
🔍 Processing reserve 1/55...
🔍 Processing reserve 11/55...
🔍 Processing reserve 21/55...
🔍 Processing reserve 31/55...
🔍 Processing reserve 41/55...
🔍 Processing reserve 51/55...
✅ Successfully processed 55/55 reserves
🎯 Risk data calculated for 55 reserves

📊 ETHEREUM RISK SUMMARY:
• Reserves: 55
• Highest borrow rate: 29.35%
• Average LTV: 41.0%
• Borrowing enabled: 35
• Frozen reserves: 3
🚨 High borrow rate: LUSD, SNX

🔄 PROCESSING POLYGON
🔗 Connecting to polygon RPC...
✅ Connected to polygon at block 77203619
📋 Getting reserves list...
📊 Found 21 reserves
🔍 Processing reserve 1/21...
🔍 Processing reserve 11/21...
🔍 Processing reserve 21/21...
✅ Successfully processed 21/21 reserves
🎯 Risk data calculated for 21 reserves

📊 POLYGON RISK SUMMARY:
• Reserves: 21
• Highest borrow rate: 144.86%
•

In [34]:
def analyze_specific_risks(chain_data):
    """Deep dive into specific risk patterns"""
    
    print(f"\n🔍 DEEP DIVE RISK ANALYSIS")
    print("=" * 80)
    
    # 1. Zero LTV but high borrow rate analysis
    print("\n📌 ZERO LTV / HIGH BORROW ASSETS (Most Dangerous):")
    for chain, df in chain_data.items():
        zero_ltv_high_borrow = df[(df['ltv'] == 0) & (df['variableBorrowRate'] > 0.05)]
        if not zero_ltv_high_borrow.empty:
            for _, asset in zero_ltv_high_borrow.iterrows():
                print(f"   • {chain:>10}: {asset['symbol']:<8} - {asset['variableBorrowRate']:>6.1%} borrow rate")
    
    # 2. High utilization analysis
    print("\n📌 HIGH UTILIZATION RESERVES (>80%):")
    for chain, df in chain_data.items():
        if 'utilization_rate' in df.columns:
            high_util = df[df['utilization_rate'] > 0.80]
            if not high_util.empty:
                for _, asset in high_util.iterrows():
                    print(f"   • {chain:>10}: {asset['symbol']:<8} - {asset['utilization_rate']:>5.1%} utilization")
    
    # 3. Cross-chain comparison of same assets
    print("\n📌 CROSS-CHAIN ASSET COMPARISON:")
    common_assets = ['USDC', 'USDT', 'WETH', 'WBTC']
    for asset in common_assets:
        rates = []
        for chain, df in chain_data.items():
            asset_data = df[df['symbol'] == asset]
            if not asset_data.empty:
                borrow_rate = asset_data.iloc[0]['variableBorrowRate']
                rates.append(f"{chain}: {borrow_rate:.1%}")
        if rates:
            print(f"   • {asset}: {', '.join(rates)}")

# Run deep analysis
analyze_specific_risks(all_data)


🔍 DEEP DIVE RISK ANALYSIS

📌 ZERO LTV / HIGH BORROW ASSETS (Most Dangerous):
   •   ethereum: LUSD     -  29.3% borrow rate
   •   ethereum: SNX      -  12.7% borrow rate
   •   ethereum: GHO      -   6.0% borrow rate
   •   ethereum: RPL      -   6.5% borrow rate
   •    polygon: DAI      -   8.7% borrow rate
   •    polygon: USDC     -   8.8% borrow rate
   •    polygon: SUSHI    -   6.3% borrow rate
   •    polygon: BAL      - 144.9% borrow rate
   •    polygon: DPI      -   8.0% borrow rate
   •    polygon: EURS     -   8.8% borrow rate
   •   arbitrum: LUSD     -   7.7% borrow rate
   •   arbitrum: FRAX     -   5.2% borrow rate
   •   arbitrum: GHO      -   6.3% borrow rate
   •   optimism: sUSD     -   5.5% borrow rate
   •   optimism: LUSD     -   7.1% borrow rate
   •  avalanche: FRAX     -  39.0% borrow rate
   •  avalanche: MAI      - 125.8% borrow rate

📌 HIGH UTILIZATION RESERVES (>80%):
   •    polygon: BAL      - 91.5% utilization

📌 CROSS-CHAIN ASSET COMPARISON:
   • US

In [38]:
def create_risk_summary_csv(chain_data):
    """Create a simple CSV summary that's guaranteed to work"""
    
    summary_data = []
    
    for chain, df in chain_data.items():
        if not df.empty:
            for _, reserve in df.iterrows():
                summary_data.append({
                    'chain': chain,
                    'symbol': str(reserve['symbol']),
                    'borrow_rate': float(reserve['variableBorrowRate']),
                    'ltv': float(reserve['ltv']),
                    'liquidation_threshold': float(reserve['liquidationThreshold']),
                    'is_frozen': bool(reserve['isFrozen']),
                    'borrowing_enabled': bool(reserve['borrowingEnabled']),
                    'risk_level': 'CRITICAL' if reserve['variableBorrowRate'] > 0.25 else 
                                 'HIGH' if reserve['variableBorrowRate'] > 0.10 else 
                                 'MEDIUM' if reserve['variableBorrowRate'] > 0.05 else 'LOW'
                })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv('aave_risk_summary.csv', index=False)
    print(f"💾 Risk summary exported to 'aave_risk_summary.csv'")
    return summary_df

# Create the summary
risk_summary = create_risk_summary_csv(all_data)
print(f"\n📋 Summary created with {len(risk_summary)} reserve entries")

💾 Risk summary exported to 'aave_risk_summary.csv'

📋 Summary created with 125 reserve entries


In [40]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

def export_risk_data(chain_data, risk_alerts):
    """Export data for integration with your early-warning system"""
    
    def convert_to_serializable(obj):
        """Convert pandas/numpy types to Python native types for JSON serialization"""
        if pd.isna(obj):
            return None
        elif isinstance(obj, (pd.Int64Dtype, pd.Int32Dtype, pd.Int16Dtype, pd.Int8Dtype)):
            return int(obj) if not pd.isna(obj) else None
        elif isinstance(obj, (pd.Float64Dtype, pd.Float32Dtype)):
            return float(obj) if not pd.isna(obj) else None
        elif isinstance(obj, (int, np.integer)):
            return int(obj)
        elif isinstance(obj, (float, np.floating)):
            return float(obj)
        elif isinstance(obj, (bool, np.bool_)):
            return bool(obj)
        else:
            return obj

    # Export to JSON for web dashboard
    export_data = {
        'timestamp': datetime.now().isoformat(),
        'summary': {
            'total_chains': len(chain_data),
            'total_reserves': sum(len(df) for df in chain_data.values()),
            'high_risk_count': len([a for a in risk_alerts if a['level'] in ['CRITICAL', 'HIGH']])
        },
        'alerts': [],
        'chains': {}
    }
    
    # Convert alerts to serializable format
    for alert in risk_alerts:
        serializable_alert = {}
        for key, value in alert.items():
            serializable_alert[key] = convert_to_serializable(value)
        export_data['alerts'].append(serializable_alert)
    
    # Convert chain data to serializable format
    for chain, df in chain_data.items():
        if not df.empty:
            # Convert DataFrame to dictionary with serializable values
            chain_data_dict = {}
            for col in df.columns:
                if col in ['symbol', 'name', 'address', 'aTokenAddress', 'variableDebtTokenAddress']:
                    chain_data_dict[col] = df[col].astype(str).tolist()
                else:
                    chain_data_dict[col] = [convert_to_serializable(x) for x in df[col]]
            
            export_data['chains'][chain] = {
                'reserve_count': len(df),
                'high_risk_assets': df[df['variableBorrowRate'] > 0.10]['symbol'].astype(str).tolist(),
                'frozen_count': int(df['isFrozen'].sum()),
                'avg_ltv': float(df['ltv'].mean()),
                'max_borrow_rate': float(df['variableBorrowRate'].max()),
                'reserves_data': chain_data_dict
            }
    
    with open('aave_risk_dashboard.json', 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"\n💾 Risk data exported to 'aave_risk_dashboard.json'")
    print("   Ready for integration with your early-warning system!")
    
    return export_data

# Alternative: Simple export that definitely works
def export_simple_risk_data(chain_data, risk_alerts):
    """Simple export that avoids serialization issues"""
    
    export_data = {
        'timestamp': datetime.now().isoformat(),
        'summary': {
            'total_chains': len(chain_data),
            'total_reserves': sum(len(df) for df in chain_data.values()),
            'high_risk_count': len([a for a in risk_alerts if a['level'] in ['CRITICAL', 'HIGH']])
        },
        'critical_alerts': [],
        'chain_summaries': {}
    }
    
    # Export only critical alerts with simple types
    for alert in risk_alerts:
        if alert['level'] in ['CRITICAL', 'HIGH']:
            simple_alert = {
                'level': str(alert['level']),
                'chain': str(alert['chain']),
                'symbol': str(alert['symbol']),
                'type': str(alert['type']),
                'value': str(alert['value'])
            }
            export_data['critical_alerts'].append(simple_alert)
    
    # Export chain summaries with explicit type conversion
    for chain, df in chain_data.items():
        if not df.empty:
            export_data['chain_summaries'][chain] = {
                'reserve_count': int(len(df)),
                'high_risk_assets': [str(x) for x in df[df['variableBorrowRate'] > 0.10]['symbol'].tolist()],
                'frozen_count': int(df['isFrozen'].sum()),
                'avg_ltv': float(df['ltv'].mean()),
                'max_borrow_rate': float(df['variableBorrowRate'].max())
            }
    
    with open('aave_risk_simple.json', 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"💾 Simple risk data exported to 'aave_risk_simple.json'")
    return export_data

# Try the simple export first
try:
    export_simple_risk_data(all_data, risk_monitor.risk_alerts)
except Exception as e:
    print(f"❌ Export failed: {e}")
    
    # Last resort: CSV export
    print("\n🔄 Falling back to CSV export...")
    for chain, df in all_data.items():
        if not df.empty:
            # Convert all columns to strings to avoid serialization issues
            csv_df = df.astype(str)
            filename = f"aave_v3_{chain}_risk.csv"
            csv_df.to_csv(filename, index=False)
            print(f"💾 {chain} data saved to {filename}")

💾 Simple risk data exported to 'aave_risk_simple.json'


In [41]:
class AaveRiskMonitor:
    def __init__(self, chain_data):
        self.chain_data = chain_data
        self.risk_alerts = []
        
    def generate_risk_alerts(self):
        """Generate prioritized risk alerts for early-warning system"""
        
        print(f"\n🔴 REAL-TIME RISK ALERTS - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 80)
        
        # Critical risk thresholds
        CRITICAL_BORROW_RATE = 0.25  # 25%
        HIGH_BORROW_RATE = 0.10      # 10%
        LOW_SAFETY_MARGIN = 0.05     # 5%
        
        for chain, df in self.chain_data.items():
            if df.empty:
                continue
                
            # Convert to native Python types for alerts
            df_clean = df.copy()
            for col in df_clean.select_dtypes(include=['float64', 'int64']).columns:
                df_clean[col] = df_clean[col].apply(lambda x: float(x) if pd.notna(x) else 0.0)
            
            # 1. Extreme borrow rate alerts
            extreme_borrow = df_clean[df_clean['variableBorrowRate'] > CRITICAL_BORROW_RATE]
            for _, reserve in extreme_borrow.iterrows():
                self.risk_alerts.append({
                    'level': 'CRITICAL',
                    'chain': chain,
                    'symbol': str(reserve['symbol']),
                    'type': 'Extreme Borrow Rate',
                    'value': f"{float(reserve['variableBorrowRate']):.1%}",
                    'details': f"LTV: {float(reserve['ltv']):.1%}, Frozen: {bool(reserve['isFrozen'])}"
                })
            
            # 2. High borrow rate alerts
            high_borrow = df_clean[(df_clean['variableBorrowRate'] > HIGH_BORROW_RATE) & 
                                  (df_clean['variableBorrowRate'] <= CRITICAL_BORROW_RATE)]
            for _, reserve in high_borrow.iterrows():
                self.risk_alerts.append({
                    'level': 'HIGH',
                    'chain': chain,
                    'symbol': str(reserve['symbol']),
                    'type': 'High Borrow Rate', 
                    'value': f"{float(reserve['variableBorrowRate']):.1%}",
                    'details': f"LTV: {float(reserve['ltv']):.1%}"
                })
            
            # 3. Low safety margin alerts
            collateral_assets = df_clean[df_clean['ltv'] > 0]
            low_safety = collateral_assets[collateral_assets['safety_margin'] < LOW_SAFETY_MARGIN]
            for _, reserve in low_safety.iterrows():
                self.risk_alerts.append({
                    'level': 'MEDIUM', 
                    'chain': chain,
                    'symbol': str(reserve['symbol']),
                    'type': 'Low Safety Margin',
                    'value': f"{float(reserve['safety_margin']):.1%}",
                    'details': f"LTV: {float(reserve['ltv']):.1%}, Liq: {float(reserve['liquidationThreshold']):.1%}"
                })
    
    def print_risk_dashboard(self):
        """Print formatted risk dashboard"""
        
        # Sort alerts by severity level
        severity_order = {'CRITICAL': 0, 'HIGH': 1, 'MEDIUM': 2, 'INFO': 3}
        self.risk_alerts.sort(key=lambda x: severity_order[x['level']])
        
        print(f"\n📊 RISK DASHBOARD - Prioritized Alerts")
        print("=" * 100)
        print(f"{'Level':<10} {'Chain':<12} {'Asset':<8} {'Type':<20} {'Value':<12} {'Details'}")
        print("-" * 100)
        
        for alert in self.risk_alerts:
            level_color = {
                'CRITICAL': '🔴',
                'HIGH': '🟠', 
                'MEDIUM': '🟡',
                'INFO': '🔵'
            }
            print(f"{level_color[alert['level']]} {alert['level']:<8} {alert['chain']:<12} "
                  f"{alert['symbol']:<8} {alert['type']:<20} {alert['value']:<12} {alert['details']}")

# Initialize and run the fixed version
risk_monitor = AaveRiskMonitor(all_data)
risk_monitor.generate_risk_alerts()
risk_monitor.print_risk_dashboard()

# Export with the simple method
export_simple_risk_data(all_data, risk_monitor.risk_alerts)


🔴 REAL-TIME RISK ALERTS - 2025-10-03 11:48:37

📊 RISK DASHBOARD - Prioritized Alerts
Level      Chain        Asset    Type                 Value        Details
----------------------------------------------------------------------------------------------------
🔴 CRITICAL ethereum     LUSD     Extreme Borrow Rate  29.3%        LTV: 0.0%, Frozen: False
🔴 CRITICAL polygon      BAL      Extreme Borrow Rate  144.9%       LTV: 0.0%, Frozen: True
🔴 CRITICAL avalanche    FRAX     Extreme Borrow Rate  39.0%        LTV: 0.0%, Frozen: False
🔴 CRITICAL avalanche    MAI      Extreme Borrow Rate  125.8%       LTV: 0.0%, Frozen: True
🟠 HIGH     ethereum     SNX      High Borrow Rate     12.7%        LTV: 0.0%
🟠 HIGH     avalanche    EURC     High Borrow Rate     10.2%        LTV: 75.0%
🟡 MEDIUM   ethereum     WETH     Low Safety Margin    2.5%         LTV: 80.5%, Liq: 83.0%
🟡 MEDIUM   ethereum     wstETH   Low Safety Margin    2.5%         LTV: 78.5%, Liq: 81.0%
🟡 MEDIUM   ethereum     USDC     Low 

{'timestamp': '2025-10-03T11:48:37.153187',
 'summary': {'total_chains': 5, 'total_reserves': 125, 'high_risk_count': 6},
 'critical_alerts': [{'level': 'CRITICAL',
   'chain': 'ethereum',
   'symbol': 'LUSD',
   'type': 'Extreme Borrow Rate',
   'value': '29.3%'},
  {'level': 'CRITICAL',
   'chain': 'polygon',
   'symbol': 'BAL',
   'type': 'Extreme Borrow Rate',
   'value': '144.9%'},
  {'level': 'CRITICAL',
   'chain': 'avalanche',
   'symbol': 'FRAX',
   'type': 'Extreme Borrow Rate',
   'value': '39.0%'},
  {'level': 'CRITICAL',
   'chain': 'avalanche',
   'symbol': 'MAI',
   'type': 'Extreme Borrow Rate',
   'value': '125.8%'},
  {'level': 'HIGH',
   'chain': 'ethereum',
   'symbol': 'SNX',
   'type': 'High Borrow Rate',
   'value': '12.7%'},
  {'level': 'HIGH',
   'chain': 'avalanche',
   'symbol': 'EURC',
   'type': 'High Borrow Rate',
   'value': '10.2%'}],
 'chain_summaries': {'ethereum': {'reserve_count': 55,
   'high_risk_assets': ['LUSD', 'SNX'],
   'frozen_count': 3,
   '

In [51]:
# Test single chain
from rpc_reserve_fetcher import AaveRPCReserveFetcher

fetcher = AaveRPCReserveFetcher()
df = fetcher.fetch_chain_reserves("ethereum")
print(f"Fetched {len(df)} reserves")
print(df[['token_symbol', 'supply_apy', 'borrow_apy', 'ltv']].head())

2025-10-03 23:01:23,885 INFO rpc_reserve_fetcher Connected to ethereum at block 23500103
2025-10-03 23:01:25,296 INFO rpc_reserve_fetcher ethereum: Found 55 reserves
2025-10-03 23:02:40,417 INFO rpc_reserve_fetcher ethereum: Processed 10/55 reserves
2025-10-03 23:02:57,303 WARNING rpc_reserve_fetcher Failed to get token info for 0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2: Could not decode contract function call to symbol() with return data: b'MKR\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', output_types: ['string']
2025-10-03 23:03:58,421 INFO rpc_reserve_fetcher ethereum: Processed 20/55 reserves
2025-10-03 23:05:29,534 INFO rpc_reserve_fetcher ethereum: Processed 30/55 reserves
2025-10-03 23:07:07,185 INFO rpc_reserve_fetcher ethereum: Processed 40/55 reserves
2025-10-03 23:08:34,707 INFO rpc_reserve_fetcher ethereum: Processed 50/55 reserves
2025-10-03 23:09:31,361 INFO rpc_reserve_fetcher ethereum: Successfull

Fetched 55 reserves
  token_symbol  supply_apy  borrow_apy    ltv
0         WETH  108.231159    2.311234  0.805
1       wstETH  101.108368    0.207231  0.785
2         WBTC  102.391547    0.210963  0.730
3         USDC  119.600843    5.353713  0.750
4          DAI  121.006574    6.241873  0.630


In [50]:
from rpc_reserve_fetcher import AaveRPCReserveFetcher
from price_fetcher import EnhancedPriceFetcher
import os

price_fetcher = EnhancedPriceFetcher(api_key=os.getenv("COINGECKO_API_KEY"))
rpc_fetcher = AaveRPCReserveFetcher(price_fetcher=price_fetcher)

df = rpc_fetcher.fetch_chain_reserves("polygon")
print(df[['token_symbol', 'price_usd', 'price_available']].head())
# Should show price_usd and price_available columns with actual data

EnhancedPriceFetcher initialized. Coingecko Pro: YES
Base URL: https://pro-api.coingecko.com/api/v3, cache_ttl=300s, request_delay=0.45s


2025-10-03 23:00:38,608 INFO rpc_reserve_fetcher Connected to polygon at block 77224581
2025-10-03 23:00:39,674 INFO rpc_reserve_fetcher polygon: Found 21 reserves


KeyboardInterrupt: 

In [52]:
from web3 import Web3
import pandas as pd
import requests

def fetch_complete_aave_reserves_working(chain: str) -> pd.DataFrame:
    """Complete working version with all risk parameters"""
    
    rpc_endpoints = {
        'ethereum': 'https://eth.llamarpc.com',
        'polygon': 'https://polygon-rpc.com',
        'arbitrum': 'https://arb1.arbitrum.io/rpc',
        'optimism': 'https://mainnet.optimism.io',
        'avalanche': 'https://api.avax.network/ext/bc/C/rpc',
        'zksync': 'https://mainnet.era.zksync.io',
        'celo': 'https://forno.celo.org',
        'gnosis': 'https://rpc.gnosis.gateway.fm',
        'sonic': 'https://rpc.soniclabs.com',
        'base': 'https://mainnet.base.org',
        'fantom': 'https://rpc.ftm.tools'
    }
    
    if chain not in rpc_endpoints:
        print(f"⚠️ Chain {chain} not supported")
        return pd.DataFrame()
    
    try:
        print(f"🔗 Connecting to {chain} RPC...")
        w3 = Web3(Web3.HTTPProvider(rpc_endpoints[chain]))
        
        if not w3.is_connected():
            print("❌ Failed to connect to RPC")
            return pd.DataFrame()
            
        print(f"✅ Connected to {chain} at block {w3.eth.block_number}")

        # Aave V3 Pool contract addresses
        pool_addresses = {
            'ethereum': '0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2',
            'polygon': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',
            'arbitrum': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',
            'optimism': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',
            'avalanche': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',
            'zksync': '0x2cD461dE3F43B77E7F6D9e5e58bc4b06f0509336',  # zkSync Era Aave V3 Pool
            'celo': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',  # Same as other chains
            'gnosis': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',  # Same as other chains
            'sonic': '0x794a61358D6845594F94dc1DB02A252b5b4814aD',  # Same as other chains
            'base': '0xA238Dd80C259a72e81d7e4664a9801593F98d1c5',  # Base Aave V3 Pool
            'fantom': '0x794a61358D6845594F94dc1DB02A252b5b4814aD'   # Same as other chains
        }
        
        # ABI for getReservesList and getReserveData only
        pool_abi = [
            {
                "inputs": [],
                "name": "getReservesList",
                "outputs": [{"internalType": "address[]", "name": "", "type": "address[]"}],
                "stateMutability": "view",
                "type": "function"
            },
            {
                "inputs": [{"internalType": "address", "name": "asset", "type": "address"}],
                "name": "getReserveData",
                "outputs": [
                    {"internalType": "uint256", "name": "configuration", "type": "uint256"},
                    {"internalType": "uint128", "name": "liquidityIndex", "type": "uint128"},
                    {"internalType": "uint128", "name": "variableBorrowIndex", "type": "uint128"},
                    {"internalType": "uint128", "name": "currentLiquidityRate", "type": "uint128"},
                    {"internalType": "uint128", "name": "currentVariableBorrowRate", "type": "uint128"},
                    {"internalType": "uint128", "name": "currentStableBorrowRate", "type": "uint128"},
                    {"internalType": "uint40", "name": "lastUpdateTimestamp", "type": "uint40"},
                    {"internalType": "uint16", "name": "id", "type": "uint16"},
                    {"internalType": "address", "name": "aTokenAddress", "type": "address"},
                    {"internalType": "address", "name": "stableDebtTokenAddress", "type": "address"},
                    {"internalType": "address", "name": "variableDebtTokenAddress", "type": "address"},
                    {"internalType": "address", "name": "interestRateStrategyAddress", "type": "address"},
                    {"internalType": "uint128", "name": "accruedToTreasury", "type": "uint128"},
                    {"internalType": "uint128", "name": "unbacked", "type": "uint128"},
                    {"internalType": "uint128", "name": "isolationModeTotalDebt", "type": "uint128"}
                ],
                "stateMutability": "view",
                "type": "function"
            }
        ]
        
        pool_address = pool_addresses[chain]
        pool_contract = w3.eth.contract(
            address=Web3.to_checksum_address(pool_address),
            abi=pool_abi
        )
        
        # Get list of reserves
        print("📋 Getting reserves list...")
        reserves_list = pool_contract.functions.getReservesList().call()
        print(f"📊 Found {len(reserves_list)} reserves")
        
        # Get token symbols and decimals
        def get_token_info(address):
            try:
                erc20_abi = [
                    {"constant": True, "inputs": [], "name": "symbol", "outputs": [{"name": "", "type": "string"}], "type": "function"},
                    {"constant": True, "inputs": [], "name": "decimals", "outputs": [{"name": "", "type": "uint8"}], "type": "function"},
                    {"constant": True, "inputs": [], "name": "name", "outputs": [{"name": "", "type": "string"}], "type": "function"}
                ]
                token_contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=erc20_abi)
                symbol = token_contract.functions.symbol().call()
                decimals = token_contract.functions.decimals().call()
                name = token_contract.functions.name().call()
                return symbol, decimals, name
            except:
                return "UNKNOWN", 18, "Unknown"
        
        # Get data for all reserves
        reserves_data = []
        successful_reserves = 0
        
        for i, reserve in enumerate(reserves_list):
            try:
                if i % 10 == 0:
                    print(f"🔍 Processing reserve {i+1}/{len(reserves_list)}...")
                
                # Get reserve data
                reserve_data = pool_contract.functions.getReserveData(reserve).call()
                
                # Get token info
                symbol, decimals, name = get_token_info(reserve)
                
                # Extract configuration data (bitmask)
                configuration = reserve_data[0]
                
                # Decode configuration (bitmask positions for Aave V3)
                # Reference: https://docs.aave.com/developers/core-contracts/pool#getreservedata
                is_active = (configuration >> 56) & 1 == 1
                is_frozen = (configuration >> 57) & 1 == 1
                borrowing_enabled = (configuration >> 58) & 1 == 1
                stable_borrowing_enabled = (configuration >> 59) & 1 == 1
                
                # Decode LTV, liquidation threshold, and liquidation bonus from configuration
                ltv = (configuration & ((1 << 16) - 1)) / 10000  # bits 0-15
                liquidation_threshold = ((configuration >> 16) & ((1 << 16) - 1)) / 10000  # bits 16-31
                liquidation_bonus = ((configuration >> 32) & ((1 << 16) - 1)) / 10000  # bits 32-47
                
                reserve_info = {
                    'address': reserve,
                    'symbol': symbol,
                    'name': name,
                    'decimals': decimals,
                    'liquidityRate': reserve_data[3] / 1e27,  # APY
                    'variableBorrowRate': reserve_data[4] / 1e27,  # APY
                    'stableBorrowRate': reserve_data[5] / 1e27,  # APY
                    'ltv': ltv,
                    'liquidationThreshold': liquidation_threshold,
                    'liquidationBonus': liquidation_bonus,
                    'isActive': is_active,
                    'isFrozen': is_frozen,
                    'borrowingEnabled': borrowing_enabled,
                    'stableBorrowingEnabled': stable_borrowing_enabled,
                    'liquidityIndex': reserve_data[1] / 1e27,
                    'variableBorrowIndex': reserve_data[2] / 1e27,
                    'lastUpdateTimestamp': reserve_data[6],
                    'aTokenAddress': reserve_data[8],
                    'variableDebtTokenAddress': reserve_data[10]
                }
                reserves_data.append(reserve_info)
                successful_reserves += 1
                
            except Exception as e:
                print(f"⚠️ Failed to get data for reserve {reserve}: {e}")
                continue
        
        df = pd.DataFrame(reserves_data)
        print(f"✅ Successfully processed {successful_reserves}/{len(reserves_list)} reserves")
        
        if successful_reserves > 0:
            # Calculate risk metrics
            df['utilization_rate'] = df['variableBorrowRate'] / (df['liquidityRate'] + 1e-18)
            df['safety_margin'] = df['liquidationThreshold'] - df['ltv']
            df['borrow_apy'] = df['variableBorrowRate'] * 100
            df['supply_apy'] = df['liquidityRate'] * 100
            
            print(f"🎯 Risk data calculated for {len(df)} reserves")
        
        return df
        
    except Exception as e:
        print(f"❌ RPC method failed: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

# Test the working version
if __name__ == "__main__":
    print("🚀 Fetching Aave V3 reserves with complete risk parameters...")
    
    # Test all chains
    chains = [
        'ethereum', 'polygon', 'arbitrum', 'optimism', 'avalanche',
        'zksync', 'celo', 'gnosis', 'sonic', 'base', 'fantom'
    ]
    
    for chain in chains:
        print(f"\n{'='*60}")
        print(f"🔄 Testing {chain.upper()}...")
        print(f"{'='*60}")
        
        df = fetch_complete_aave_reserves_working(chain)
        
        if not df.empty:
            print(f"\n🎯 {chain.upper()} - COMPLETE RISK MONITORING DATA - {len(df)} RESERVES")
            
            # Display comprehensive risk metrics
            risk_columns = [
                'symbol', 'liquidityRate', 'variableBorrowRate', 'ltv', 
                'liquidationThreshold', 'utilization_rate', 'safety_margin',
                'isActive', 'borrowingEnabled'
            ]
            
            print(df[risk_columns].head(10).to_string(index=False, float_format='%.4f'))
            
            print(f"\n📊 {chain.upper()} RISK SUMMARY:")
            print(f"• Total reserves: {len(df)}")
            print(f"• Highest borrow rate: {df['variableBorrowRate'].max():.2%}")
            print(f"• Average LTV: {df['ltv'].mean():.1%}")
            print(f"• Average liquidation threshold: {df['liquidationThreshold'].mean():.1%}")
            print(f"• Reserves with borrowing enabled: {df['borrowingEnabled'].sum()}")
            print(f"• Frozen reserves: {df['isFrozen'].sum()}")
            
            # Identify high-risk reserves
            high_borrow_risk = df[df['variableBorrowRate'] > 0.10]  # > 10% borrow rate
            if not high_borrow_risk.empty:
                print(f"\n🚨 HIGH BORROW RATE RESERVES (>10%):")
                for _, reserve in high_borrow_risk.iterrows():
                    print(f"   • {reserve['symbol']}: {reserve['variableBorrowRate']:.2%} borrow rate")
            
            # Identify low safety margin reserves
            low_safety = df[df['safety_margin'] < 0.05]  # < 5% safety margin
            if not low_safety.empty:
                print(f"\n⚠️ LOW SAFETY MARGIN RESERVES (<5% margin):")
                for _, reserve in low_safety.iterrows():
                    print(f"   • {reserve['symbol']}: {reserve['safety_margin']:.1%} margin (LTV: {reserve['ltv']:.1%}, Liq: {reserve['liquidationThreshold']:.1%})")
            
            # Identify high utilization reserves
            high_utilization = df[df['utilization_rate'] > 0.80]  # > 80% utilization
            if not high_utilization.empty:
                print(f"\n🔴 HIGH UTILIZATION RESERVES (>80%):")
                for _, reserve in high_utilization.iterrows():
                    print(f"   • {reserve['symbol']}: {reserve['utilization_rate']:.1%} utilization")
        
        else:
            print(f"❌ No data fetched for {chain}")
    
    print(f"\n{'='*60}")
    print("🎉 All chains processed!")
    print(f"{'='*60}")

🚀 Fetching Aave V3 reserves with complete risk parameters...

🔄 Testing ETHEREUM...
🔗 Connecting to ethereum RPC...
✅ Connected to ethereum at block 23504889
📋 Getting reserves list...
📊 Found 55 reserves
🔍 Processing reserve 1/55...
🔍 Processing reserve 11/55...
🔍 Processing reserve 21/55...
🔍 Processing reserve 31/55...
🔍 Processing reserve 41/55...
🔍 Processing reserve 51/55...
✅ Successfully processed 55/55 reserves
🎯 Risk data calculated for 55 reserves

🎯 ETHEREUM - COMPLETE RISK MONITORING DATA - 55 RESERVES
symbol  liquidityRate  variableBorrowRate    ltv  liquidationThreshold  utilization_rate  safety_margin  isActive  borrowingEnabled
  WETH         1.0824              0.0231 0.8050                0.8300            0.0213         0.0250      True              True
wstETH         1.0111              0.0019 0.7850                0.8100            0.0019         0.0250      True              True
  WBTC         1.0239              0.0022 0.7300                0.7800            0

Traceback (most recent call last):
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\web3\contract\utils.py", line 204, in call_contract_function
    output_data = w3.codec.decode(output_types, return_data)
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\codec.py", line 162, in decode
    return cast(Tuple[Any, ...], decoder(stream))
                                 ~~~~~~~^^^^^^^^
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\decoding.py", line 131, in __call__
    return self.decode(stream)
           ~~~~~~~~~~~^^^^^^^^
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_utils\functional.py", line 47, in inner
    return callback(fn(*args, **kwargs))
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\decoding.py", line 224, in decode
    self.validate_pointers(stream)
    ~~~~~~~~~~~~~~~~~~~~~~^

❌ No data fetched for zksync

🔄 Testing CELO...
🔗 Connecting to celo RPC...
✅ Connected to celo at block 47686349
📋 Getting reserves list...
❌ RPC method failed: Could not transact with/call contract function, is contract deployed correctly and chain synced?
❌ No data fetched for celo

🔄 Testing GNOSIS...
🔗 Connecting to gnosis RPC...


Traceback (most recent call last):
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\web3\contract\utils.py", line 204, in call_contract_function
    output_data = w3.codec.decode(output_types, return_data)
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\codec.py", line 162, in decode
    return cast(Tuple[Any, ...], decoder(stream))
                                 ~~~~~~~^^^^^^^^
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\decoding.py", line 131, in __call__
    return self.decode(stream)
           ~~~~~~~~~~~^^^^^^^^
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_utils\functional.py", line 47, in inner
    return callback(fn(*args, **kwargs))
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\decoding.py", line 224, in decode
    self.validate_pointers(stream)
    ~~~~~~~~~~~~~~~~~~~~~~^

✅ Connected to gnosis at block 42453481
📋 Getting reserves list...
❌ RPC method failed: Could not transact with/call contract function, is contract deployed correctly and chain synced?
❌ No data fetched for gnosis

🔄 Testing SONIC...
🔗 Connecting to sonic RPC...


Traceback (most recent call last):
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\web3\contract\utils.py", line 204, in call_contract_function
    output_data = w3.codec.decode(output_types, return_data)
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\codec.py", line 162, in decode
    return cast(Tuple[Any, ...], decoder(stream))
                                 ~~~~~~~^^^^^^^^
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\decoding.py", line 131, in __call__
    return self.decode(stream)
           ~~~~~~~~~~~^^^^^^^^
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_utils\functional.py", line 47, in inner
    return callback(fn(*args, **kwargs))
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\decoding.py", line 224, in decode
    self.validate_pointers(stream)
    ~~~~~~~~~~~~~~~~~~~~~~^

✅ Connected to sonic at block 49318126
📋 Getting reserves list...
❌ RPC method failed: Could not transact with/call contract function, is contract deployed correctly and chain synced?
❌ No data fetched for sonic

🔄 Testing BASE...
🔗 Connecting to base RPC...


Traceback (most recent call last):
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\web3\contract\utils.py", line 204, in call_contract_function
    output_data = w3.codec.decode(output_types, return_data)
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\codec.py", line 162, in decode
    return cast(Tuple[Any, ...], decoder(stream))
                                 ~~~~~~~^^^^^^^^
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\decoding.py", line 131, in __call__
    return self.decode(stream)
           ~~~~~~~~~~~^^^^^^^^
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_utils\functional.py", line 47, in inner
    return callback(fn(*args, **kwargs))
  File "c:\Users\g\Documents\LiquidationProject\DefiLiquidation\Lib\site-packages\eth_abi\decoding.py", line 224, in decode
    self.validate_pointers(stream)
    ~~~~~~~~~~~~~~~~~~~~~~^

✅ Connected to base at block 36398884
📋 Getting reserves list...
📊 Found 14 reserves
🔍 Processing reserve 1/14...
⚠️ Failed to get data for reserve 0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA: 429 Client Error: Too Many Requests for url: https://mainnet.base.org/
🔍 Processing reserve 11/14...
✅ Successfully processed 13/14 reserves
🎯 Risk data calculated for 13 reserves

🎯 BASE - COMPLETE RISK MONITORING DATA - 13 RESERVES
 symbol  liquidityRate  variableBorrowRate    ltv  liquidationThreshold  utilization_rate  safety_margin  isActive  borrowingEnabled
   WETH         1.0555              0.0231 0.8000                0.8300            0.0219         0.0300      True              True
  cbETH         1.0247              0.0065 0.7500                0.7900            0.0063         0.0400      True              True
 wstETH         1.0158              0.0023 0.7500                0.7900            0.0023         0.0400      True              True
UNKNOWN         1.1482              0.0759